In [1]:
#pip install transformers

     |████████████████████████████████| 1.4 MB 19.1 MB/s eta 0:00:01
     |████████████████████████████████| 883 kB 82.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 73.1 MB/s eta 0:00:01
     |████████████████████████████████| 719 kB 70.3 MB/s eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893259 sha256=618f97da88314ab00e51b2df397b170fb7685eb9b4a34fdea28315fcaf8f8527
  Stored in directory: /home/jovyan/.cache/pip/wheels/69/09/d1/bf058f7d6fa0ecba2ce7c66be3b8d012beb4bf61a6e0c101c0
Successfully built sacremoses
Note: you may need to restart the kernel to use updated packages.


In [2]:
#pip install torch torchvision

Note: you may need to restart the kernel to use updated packages.


In [3]:
#pip install --upgrade pyarrow

     |████████████████████████████████| 17.7 MB 19.7 MB/s eta 0:00:01    |████▉                           | 2.7 MB 19.7 MB/s eta 0:00:01
ERROR: cudf 0.6.1 has requirement pyarrow==0.12.1, but you'll have pyarrow 2.0.0 which is incompatible.
ERROR: cudf-cuda92 0.6.1 has requirement pyarrow==0.12.1, but you'll have pyarrow 2.0.0 which is incompatible.
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 0.12.1
    Uninstalling pyarrow-0.12.1:
      Successfully uninstalled pyarrow-0.12.1
Note: you may need to restart the kernel to use updated packages.


In [4]:
#pip install --upgrade tensorflow-gpu

     |██████████████████████████████  | 300.9 MB 109.6 MB/s eta 0:00:01     |█████████████████████▏          | 212.1 MB 116.9 MB/s eta 0:00:01

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 320.4 MB 25 kB/s 
     |████████████████████████████████| 459 kB 15.6 MB/s eta 0:00:01
     |████████████████████████████████| 10.6 MB 97.8 MB/s eta 0:00:01
     |████████████████████████████████| 779 kB 80.1 MB/s eta 0:00:01
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorflow-estimator-2.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.1.1
    Uninstalling tensorboard-2.1.1:
      Successfully uninstalled tensorboard-2.1.1
  Attempting uninstall: tensorflow-gpu
    Found existing installation: tensorflow-gpu 2.1.0
    Uninstalling tensorflow-gpu-2.1.0:
      Successfully uninstalled tensorflow-gpu-2.1.0
Note: you may need to rest

In [5]:
import torch
import pandas as pd
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
# pose sequence as a NLI premise and label as a hypothesis
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW




In [6]:
entailment_data = pd.read_csv('./gigaword1000entailment.csv')


train_texts  = list(map(list, zip(entailment_data['document'], entailment_data['summary'])))
# map to mnli mapping, entailment = 2, contradiction = 0
train_labels = [2 if label == 0 else 0 for label in entailment_data['label']]

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [7]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

train_encodings = tokenizer(train_texts, return_tensors='pt', truncation='only_first', padding =True)
val_encodings = tokenizer(val_texts, return_tensors='pt', truncation='only_first', padding =True)

In [8]:
class EntailmentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = EntailmentDataset(train_encodings, train_labels)
val_dataset = EntailmentDataset(val_encodings, val_labels)


In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')#.to(device)
if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  model = torch.nn.DataParallel(model)

model.to(device)

# for param in model.bart.bart.parameters():
#     param.requires_grad = False
# Change batch size to 32 when we have access to many gpu's
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(10):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.mean().backward()
        #loss.backward()
        optim.step()

model.eval()

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Let's use 2 GPUs!


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


DataParallel(
  (module): BartForSequenceClassification(
    (model): BartModel(
      (shared): Embedding(50265, 1024, padding_idx=1)
      (encoder): BartEncoder(
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
        (layers): ModuleList(
          (0): EncoderLayer(
            (self_attn): Attention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
            (final_layer_n

In [10]:
print(model)

DataParallel(
  (module): BartForSequenceClassification(
    (model): BartModel(
      (shared): Embedding(50265, 1024, padding_idx=1)
      (encoder): BartEncoder(
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
        (layers): ModuleList(
          (0): EncoderLayer(
            (self_attn): Attention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
            (final_layer_n

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli').to(device)
# tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')
label = "history"
premise = "Crunch Wrap Supreme"
hypothesis = f'This example is {label}.'

# run through model pre-trained on MNLI
x = tokenizer.encode(premise, hypothesis, return_tensors='pt', truncation='only_first', padding =True)
logits = model(x.to(device))[0]

# we throw away "neutral" (dim 1) and take the probability of
# "entailment" (2) as the probability of the label being true 
entail_contradiction_logits = logits[:,[0,2]]
probs = entail_contradiction_logits.softmax(dim=1)
prob_label_is_true = probs[:,1]

In [12]:
print(logits.shape)

print(entail_contradiction_logits.shape)
print(entail_contradiction_logits)

print(probs.shape)
print(probs)


print(prob_label_is_true)

torch.Size([1, 3])
torch.Size([1, 2])
tensor([[ 3.3434, -0.3393]], device='cuda:0', grad_fn=<IndexBackward>)
torch.Size([1, 2])
tensor([[0.9755, 0.0245]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([0.0245], device='cuda:0', grad_fn=<SelectBackward>)


In [13]:
print(x)

tensor([[    0, 32105, 35989,  2124,     2,     2,   713,  1246,    16,   750,
             4,     2]])


In [19]:
torch.save(model.state_dict(),"./model/kien.pth")